In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Vado a caricare i file con cui lavorerò
path = r'C:\Users\Gatti\Desktop\ProgettoSC_2-work_branch\result\ProgettoSocial\Dataframe'
workers_acl = pd.read_csv(f'{path}\\workers_acl.csv')
workers_answers = pd.read_csv(f'{path}\\workers_answers.csv')
workers_dimensions_selection = pd.read_csv(f'{path}\\workers_dimensions_selection.csv')
workers_documents = pd.read_csv(f'{path}\\workers_documents.csv')
workers_questionnaire = pd.read_csv(f'{path}\\workers_questionnaire.csv')

4.4.b

Calcolare le frequenze relative delle risposte date dai lavoratori al questionario.  
Determinare la percentuale di lavoratori che ha fornito ciascuna risposta possibile in un questionario. In altre parole, si tratta di analizzare quanti partecipanti hanno scelto una specifica opzione di risposta rispetto al totale dei partecipanti.

Un'HIT contiene 5 reviews di cui bisognava rispondere a 6 fattori diversi. Abbiamo così un totale di 30 risposte per HIT.
La stessa HIT era riproposta altre 4 volte all'interno nel progetto.

Andiamo a rimodellare i vari file in modo da lavorare solo sui dati che ci servono

In [ ]:
#Risposte dei workers
workers_answers = workers_answers[['worker_id', 'unit_id', 'action', 'document_id', 'document_index', 'reliability_value', 
                                   'neutrality_value', 'comprehensibility_value', 'precision_value', 
                                   'completeness_value', 'informativeness_value', 'confidence_value', 
                                   'confidence_justification', 'time_start', 'time_end', 'time_start_parsed', 
                                   'time_end_parsed']]
workers_answers.to_csv('ProgettoSocial\Dataframe\workers_answers.csv', index=False)

In [ ]:
#Questo file contiene informazioni relative alla confidence
workers_dimensions_selection = workers_dimensions_selection[
    ['worker_id', 'unit_id', 'document_id', 'document_index', 'dimension_name', 'selection_index', 'selection_value', 
     'selection_timestamp_parsed']]

workers_dimensions_selection.to_csv('ProgettoSocial\Dataframe\workers_dimensions_selection.csv', index=False)

In [ ]:
#Questo file contiene le info riguardo le review, in che unità sono state inserite e quali worker le hanno valutate
workers_documents = workers_documents[['document_id', 'unit_ids', 'worker_ids', 'text', 'product_title', 
                                       'product_brand', 'time']]

workers_documents.to_csv('ProgettoSocial\Dataframe\workers_documents.csv', index=False)

In [ ]:
#Questo file fornisce informazioni sul questionario.
workers_questionnaire = workers_questionnaire[['worker_id', 'unit_id', 'questionnaire_position', 'questionnaire_allow_back', 
                                               'questionnaire_type', 'questionnaire_time_elapsed', 
                                               'question_attribute_indexFull', 'question_attribute_name', 
                                               'question_attribute_type', 'question_attribute_nameFull', 
                                               'question_answer_value', 'question_answer_text']]

workers_questionnaire.to_csv('ProgettoSocial\Dataframe\workers_questionnaire.csv', index=False)

In [ ]:
#Ora andremo a creare un data frame df composto da:
#righe:worker_id (prendiamo dal file workers_acl)
#colonne: document_id (prendiamo dal file workers_documents)
righe = workers_acl['worker_id']
colonne = workers_documents['document_id']

#Creo un DataFrame con gli oggetti come indici di righe e colonne
df = pd.DataFrame(index=righe, columns=colonne)

#Questo è il risultato
display(df)

In [ ]:
#Vado a crearmi un data frame nel quale ho solo i valori delle risposte e le "coordinate"
#coordinate: worker_id X document_id
answers_values = workers_answers[['worker_id', 'document_id', 'reliability_value', 
                                   'neutrality_value', 'comprehensibility_value', 'precision_value', 
                                   'completeness_value', 'informativeness_value']]

display(answers_values)
answers_values.to_csv('ProgettoSocial\Dataframe\\answers_values.csv', index=False)

In [ ]:
#Vado a creare un vettore i cui valori sono le risposte che ogni worker ha dato ad una review
#Lo associo al worker_id
review_vector = pd.DataFrame(answers_values['document_id'])
review_vector['worker_vettore'] = answers_values.apply(lambda row: [row['worker_id'], row['reliability_value'],
                                                                     row['neutrality_value'],
                                                                     row['comprehensibility_value'],
                                                                     row['precision_value'],
                                                                     row['completeness_value'],
                                                                     row['informativeness_value']], axis=1)

#Metto in ordine il data frame review_vector
review_vector = review_vector.sort_values(by='document_id').reset_index(drop=True)

#Visualizzo il risultato
display(review_vector)

In [ ]:
#Itero attraverso le righe di reviewer_vector e inserisco i valori in df
for index, row in review_vector.iterrows():
    document_id = row['document_id']
    worker_id = row['worker_vettore'][0]
    values = row['worker_vettore'][1:7]

    # Inserisco i valori nelle celle corrispondenti in df
    df.at[worker_id, document_id] = values

#Ecco il risultato
display(df)

#Abbiamo ottenuto finalmente il nostro prezioso 'df' su cui andremo a lavorare
df.to_csv('ProgettoSocial\Dataframe\df.csv')

Analizzare quanti partecipanti hanno scelto una specifica opzione di risposta rispetto al totale dei partecipanti.  
Sarebbe opportuno prendere in considerazione solo i partecipanti che avevano quella review da analizzare e non considerare tutti i worker che hanno lavorato al progetto in quanto ognuno aveva accesso a sole 5 reviews su una base di 15 reviews.

Detto ciò, la cosa è abbastanza facile da fare: 
-selezionare una colonna ed inserirla in un nuovo data frame  
-separarne i valori in modo da avere un data frame di 6 colonne  
-andare a contare quante sono le righe del data frame  
-fare il conteggio dei valori presenti nelle colonne (colonna per colonna)  
-essendo 5 le possibili scelte di una data categoria (reliability, neutrality, comprehensibility, precision, completeness, informativeness), avremo 5 FR (frequenza relativa) per ogni colonna di un'unica review 

Tirando le somme sono 5(scelte) * 6(categorie) * 15(reviews)--->un totale di 450 FR

In [ ]:
#Estraggo i valori relativi alla reliability
reliability = pd.DataFrame()
for colonna in df.columns:
    reliability[colonna] = df[colonna].apply(lambda x: x[0] if isinstance(x, (list, str)) else x)
    
reliability.name = 'Valori della reliability'

#Visualizzo il data frame relativo alla reliability
print(reliability.name)
display(reliability)

#Calcolo la frequenza relativa
reliability_fr = reliability.apply(lambda x: x.value_counts(normalize=True).sort_index()).T

#Andiamo a sostituire i valori nulli con 0 e poi moltiplichiamo il tutto per 100
reliability_fr = reliability_fr.fillna(0)
reliability_fr = reliability_fr * 100

#Arrotondiamo a solo 2 cifre decimali
reliability_fr = reliability_fr.round(2)

#Visualizzo i valori delle frequenze relative
print("Valori delle frequenze relative per il valore reliability:")
display(reliability_fr)

#Calcolo la frequenza relativa e crea un istogramma
isto_reliability = pd.DataFrame(reliability)

plt.figure(figsize=(12, 6))
isto_reliability.apply(lambda x: x.value_counts(normalize=True).sort_index()).T.plot(kind='bar', stacked=True)
plt.title('Confronto delle Frequenze Relative per Colonne')
plt.xlabel('Valori')
plt.ylabel('Frequenza Relativa')
plt.legend(title='reliability', bbox_to_anchor=(1.05, 1), loc='upper left')

#ANDIAMO A SALVARE TUTTO PRIMA DI MOSTRARE IL GRAFICO, ALTRIMENTI SALVA UN'IMMAGINE BIANCA
#Salviamo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\reliability'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)

#Salvo il DataFrame in formato CSV nella cartella di output
reliability_fr.to_csv(os.path.join(cartella_output, 'reliability_fr.csv'))

#Salvo il grafico nella cartella di output
plt.savefig(os.path.join(cartella_output, 'isto_reliability.png'))

plt.show()

In [ ]:
#Estraggo i valori relativi alla neutrality
neutrality = pd.DataFrame()
for colonna in df.columns:
    neutrality[colonna] = df[colonna].apply(lambda x: x[1] if isinstance(x, (list, str)) else x)
    
neutrality.name = 'Valori della neutrality'

#Visualizzo il data frame relativo alla neutrality
print(neutrality.name)
display(neutrality)

#Calcolo la frequenza relativa
neutrality_fr = neutrality.apply(lambda x: x.value_counts(normalize=True).sort_index()).T

#Andiamo a sostituire i valori nulli con 0 e poi moltiplichiamo il tutto per 100
neutrality_fr = neutrality_fr.fillna(0)
neutrality_fr = neutrality_fr * 100

#Arrotondiamo a solo 2 cifre decimali
neutrality_fr = neutrality_fr.round(2)

#Visualizzo i valori delle frequenze relative
print("Valori delle frequenze relative per il valore neutrality:")
display(neutrality_fr)

#Calcolo la frequenza relativa e crea un istogramma
isto_neutrality = pd.DataFrame(neutrality)

plt.figure(figsize=(12, 6))
isto_neutrality.apply(lambda x: x.value_counts(normalize=True).sort_index()).T.plot(kind='bar', stacked=True)
plt.title('Confronto delle Frequenze Relative per Colonne')
plt.xlabel('Valori')
plt.ylabel('Frequenza Relativa')
plt.legend(title='neutrality', bbox_to_anchor=(1.05, 1), loc='upper left')

#ANDIAMO A SALVARE TUTTO PRIMA DI MOSTRARE IL GRAFICO, ALTRIMENTI SALVA UN'IMMAGINE BIANCA
#Salviamo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\neutrality'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)

#Salvo il DataFrame in formato CSV nella cartella di output
neutrality_fr.to_csv(os.path.join(cartella_output, 'neutrality_fr.csv'))

#Salvo il grafico nella cartella di output
plt.savefig(os.path.join(cartella_output, 'isto_neutrality.png'))

plt.show()

In [ ]:
#Estraggo i valori relativi alla comprehensibility
comprehensibility = pd.DataFrame()
for colonna in df.columns:
    comprehensibility[colonna] = df[colonna].apply(lambda x: x[2] if isinstance(x, (list, str)) else x)
    
comprehensibility.name = 'Valori della comprehensibility'

#Visualizzo il data frame relativo alla comprehensibility
print(comprehensibility.name)
display(comprehensibility)

#Calcolo la frequenza relativa
comprehensibility_fr = comprehensibility.apply(lambda x: x.value_counts(normalize=True).sort_index()).T

#Andiamo a sostituire i valori nulli con 0 e poi moltiplichiamo il tutto per 100
comprehensibility_fr = comprehensibility_fr.fillna(0)
comprehensibility_fr = comprehensibility_fr * 100

#Arrotondiamo a solo 2 cifre decimali
comprehensibility_fr = comprehensibility_fr.round(2)

#Visualizzo i valori delle frequenze relative
print("Valori delle frequenze relative per il valore comprehensibility:")
display(comprehensibility_fr)

#Calcolo la frequenza relativa e crea un istogramma
isto_comprehensibility = pd.DataFrame(comprehensibility)

plt.figure(figsize=(12, 6))
isto_comprehensibility.apply(lambda x: x.value_counts(normalize=True).sort_index()).T.plot(kind='bar', stacked=True)
plt.title('Confronto delle Frequenze Relative per Colonne')
plt.xlabel('Valori')
plt.ylabel('Frequenza Relativa')
plt.legend(title='comprehensibility', bbox_to_anchor=(1.05, 1), loc='upper left')

#ANDIAMO A SALVARE TUTTO PRIMA DI MOSTRARE IL GRAFICO, ALTRIMENTI SALVA UN'IMMAGINE BIANCA
#Salviamo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\comprehensibility'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)

#Salvo il DataFrame in formato CSV nella cartella di output
comprehensibility_fr.to_csv(os.path.join(cartella_output, 'comprehensibility_fr.csv'))

#Salvo il grafico nella cartella di output
plt.savefig(os.path.join(cartella_output, 'isto_comprehensibility.png'))

plt.show()

In [ ]:
#Estraggo i valori relativi alla precision
precision = pd.DataFrame()
for colonna in df.columns:
    precision[colonna] = df[colonna].apply(lambda x: x[3] if isinstance(x, (list, str)) else x)
    
precision.name = 'Valori della precision'

#Visualizzo il data frame relativo alla precision
print(precision.name)
display(precision)

#CalcolO la frequenza relativa
precision_fr = precision.apply(lambda x: x.value_counts(normalize=True).sort_index()).T

#Andiamo a sostituire i valori nulli con 0 e poi moltiplichiamo il tutto per 100
precision_fr = precision_fr.fillna(0)
precision_fr = precision_fr * 100

#Arrotondiamo a solo 2 cifre decimali
precision_fr = precision_fr.round(2)

#Visualizzo i valori delle frequenze relative
print("Valori delle frequenze relative per il valore precision:")
display(precision_fr)

#Calcolo la frequenza relativa e crea un istogramma
isto_precision = pd.DataFrame(precision)

plt.figure(figsize=(12, 6))
isto_precision.apply(lambda x: x.value_counts(normalize=True).sort_index()).T.plot(kind='bar', stacked=True)
plt.title('Confronto delle Frequenze Relative per Colonne')
plt.xlabel('Valori')
plt.ylabel('Frequenza Relativa')
plt.legend(title='precision', bbox_to_anchor=(1.05, 1), loc='upper left')

#ANDIAMO A SALVARE TUTTO PRIMA DI MOSTRARE IL GRAFICO, ALTRIMENTI SALVA UN'IMMAGINE BIANCA
#Salviamo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\precision'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)

#Salvo il DataFrame in formato CSV nella cartella di output
precision_fr.to_csv(os.path.join(cartella_output, 'precision_fr.csv'))

#Salvo il grafico nella cartella di output
plt.savefig(os.path.join(cartella_output, 'isto_precision.png'))

plt.show()

In [ ]:
#Estraggo i valori relativi alla completeness
completeness = pd.DataFrame()
for colonna in df.columns:
    completeness[colonna] = df[colonna].apply(lambda x: x[4] if isinstance(x, (list, str)) else x)
    
completeness.name = 'Valori della completeness'

#Visualizzo il data frame relativo alla completeness
print(completeness.name)
display(completeness)

#Calcolo la frequenza relativa
completeness_fr = completeness.apply(lambda x: x.value_counts(normalize=True).sort_index()).T

#Andiamo a sostituire i valori nulli con 0 e poi moltiplichiamo il tutto per 100
completeness_fr = completeness_fr.fillna(0)
completeness_fr = completeness_fr * 100

#Arrotondiamo a solo 2 cifre decimali
completeness_fr = completeness_fr.round(2)

#Visualizzo i valori delle frequenze relative
print("Valori delle frequenze relative per il valore completeness:")
display(completeness_fr)

#Calcolo la frequenza relativa e crea un istogramma
isto_completeness = pd.DataFrame(completeness)

plt.figure(figsize=(12, 6))
isto_completeness.apply(lambda x: x.value_counts(normalize=True).sort_index()).T.plot(kind='bar', stacked=True)
plt.title('Confronto delle Frequenze Relative per Colonne')
plt.xlabel('Valori')
plt.ylabel('Frequenza Relativa')
plt.legend(title='completeness', bbox_to_anchor=(1.05, 1), loc='upper left')

#ANDIAMO A SALVARE TUTTO PRIMA DI MOSTRARE IL GRAFICO, ALTRIMENTI SALVA UN'IMMAGINE BIANCA
#Salviamo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\completeness'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)

#Salvo il DataFrame in formato CSV nella cartella di output
completeness_fr.to_csv(os.path.join(cartella_output, 'completeness_fr.csv'))

#Salvo il grafico nella cartella di output
plt.savefig(os.path.join(cartella_output, 'isto_completeness.png'))

plt.show()

In [ ]:
#Estraggo i valori relativi alla informativeness
informativeness = pd.DataFrame()
for colonna in df.columns:
    informativeness[colonna] = df[colonna].apply(lambda x: x[5] if isinstance(x, (list, str)) else x)
    
informativeness.name = 'Valori della informativeness'

#Visualizzo il data frame relativo alla informativeness
print(informativeness.name)
display(informativeness)

#Calcolo la frequenza relativa
informativeness_fr = informativeness.apply(lambda x: x.value_counts(normalize=True).sort_index()).T

#Andiamo a sostituire i valori nulli con 0 e poi moltiplichiamo il tutto per 100
informativeness_fr = informativeness_fr.fillna(0)
informativeness_fr = informativeness_fr * 100

#Arrotondiamo a solo 2 cifre decimali
informativeness_fr = informativeness_fr.round(2)

#Visualizzo i valori delle frequenze relative
print("Valori delle frequenze relative per il valore informativeness:")
display(informativeness_fr)

#Calcolo la frequenza relativa e crea un istogramma
isto_informativeness = pd.DataFrame(informativeness)

plt.figure(figsize=(12, 6))
isto_informativeness.apply(lambda x: x.value_counts(normalize=True).sort_index()).T.plot(kind='bar', stacked=True)
plt.title('Confronto delle Frequenze Relative per Colonne')
plt.xlabel('Valori')
plt.ylabel('Frequenza Relativa')
plt.legend(title='informativeness', bbox_to_anchor=(1.05, 1), loc='upper left')

#ANDIAMO A SALVARE TUTTO PRIMA DI MOSTRARE IL GRAFICO, ALTRIMENTI SALVA UN'IMMAGINE BIANCA
# Salviamo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\informativeness'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)

#Salvo il DataFrame in formato CSV nella cartella di output
informativeness_fr.to_csv(os.path.join(cartella_output, 'informativeness_fr.csv'))

#Salvo il grafico nella cartella di output
plt.savefig(os.path.join(cartella_output, 'isto_informativeness.png'))

plt.show()

4.4.c 

Scegliete una delle dimensioni categoriali e calcolate il percent agreement tra
worker per i valori forniti.

Qui basta scegliere una delle 6 categorie:  
reliability, neutrality, comprehensibility, precision, completeness o informativeness.

Prendiamo precision

In [ ]:
#Andiamo a rivisualizzare il data frame di precision
display(precision_fr)

Il Percent Agreement è un valore che indica quanto sono d'accordo tra di loro i worker
Si calcola: (Numero di accordi) / (Numero totale di osservazioni). Il valore ottenuto si moltiplica per 100.

Numero di accordi = è il numero di casi in cui i worker hanno risposto alla stesso modo  
Numero totale di osservazioni = è il numero totale delle risposte date dai worker

Per tradurre questa formula in modo da poter andar ad operare con i valori nel nostro data frame precision_fr  
possiamo riscrivere così:  
presa una riga (ad esempio Pair_A21VR3CU76PEA7-B003BZQRV2) avremo che  
Numero di accordi = è il valore più alto in quella riga (40.00 in questo caso)
Numero totale di osservazioni = è la somma di tutti i valori riportati in quella riga (100 in questo caso)

Quindi a noi basterà andare a cercare il valore più alto nella riga e dividerlo per la somma di tutti i valori di quella riga.  
Fatto ciò moltiplicare per 100. Avremo così il Percent Agreement

In [ ]:
#Andiamo a calcolarci il precision_percent_agreement_value
#il dataframe sarà abbreviato con ppav

ppav = pd.DataFrame()

#Funzione per ottenere il massimo valore e il nome della colonna
def massimo_e_colonna(riga):
    valore_massimo = riga.max()
    nome_colonna = riga.idxmax()
    return f'{valore_massimo} / {nome_colonna}'

#Creazione della nuova colonna multivalore
ppav['Numero_accordi-Valore_risposta'] = precision_fr.apply(massimo_e_colonna, axis=1)

#Facciamo la somma dei valori in una specifica riga (ad esempio, la prima riga)
somma_valori_riga = precision_fr.iloc[0].sum()

#Aggiungiamo una nuova colonna al DataFrame contenente la somma dei valori per ogni riga
ppav['Numero_totale_osservazioni'] = somma_valori_riga

#Dividiamo il primo valore della colonna "Numero_accordi-Valore_risposta" per il valore corrispondente della colonna "Numero_totale_osservazioni"
#e moltiplico per 100
ppav['percent_agreement'] = (ppav['Numero_accordi-Valore_risposta'].str.split('/').str[0].astype(float) / ppav['Numero_totale_osservazioni']) * 100

#Andiamo a cancellare la colonna 'Numero_accordi-Valore_risposta' per ottenere solo la colonna 'valore_risposta'
ppav['valore_risposta'] = ppav['Numero_accordi-Valore_risposta'].str.split('/').str[1].astype(float)

#Elimino la colonna "Numero_accordi-Valore_risposta"
ppav.drop(['Numero_accordi-Valore_risposta', 'Numero_totale_osservazioni'], axis=1, inplace=True)

#Visualizzo il DataFrame risultante
display(ppav)

#Salvo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\ppav'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)
    
#Salvo il DataFrame in formato CSV nella cartella di output
ppav.to_csv(os.path.join(cartella_output, 'ppav.csv'))

OSSERVAZIONE!!!

Dal data frame 'percent_agreement_value' sembra che tutti i lavoratori concordino che  
le review: Pair_AVJC3PLM2VHRI-B01D9JG0VQ e Pair_AR6PRH6B8N7LP-B006UKI83U siano estremamente precise

4.4.d 

Calcolate la confidence media espressa dai worker nel valutare ciascuna recensione

4.4.e 

Calcolate il tempo medio impiegato dai worker per valutare ciascuna recensione

In [ ]:
#Per prima cosa, mi ricopio i data frame che mi servono e li ordino per una maggiore leggibilità
time_media = workers_answers[['worker_id', 'unit_id', 'document_id', 'document_index', 'time_start', 'time_end']]

#Droppiamo 'dimension_name' e 'selection_timestamp_parsed'
confidence_media = workers_dimensions_selection.drop(['dimension_name', 'selection_timestamp_parsed'], axis=1)

#Andiamo ora a trovarci il tempo che ci hanno messo i worker sulla singola review
time_media = time_media.copy()
time_media['time_lapsed'] = time_media['time_end'] - time_media['time_start']
time_media = time_media.drop(['time_start', 'time_end'], axis=1)

display(time_media)
display(confidence_media)

In [ ]:
#Calcoliamo il tempo medio per ogni recensione in base al worker
tempo_medio = time_media.groupby(['worker_id', 'unit_id', 'document_id']).mean()['time_lapsed'].reset_index()
display(tempo_medio)

In [ ]:
#Calcoliamo la confidence_media per ogni recensione in base al worker
confidenza_media = confidence_media.groupby(['worker_id', 'unit_id', 'document_id', 'document_index']).mean()['selection_value'].reset_index()

#riordino il data frame per una maggiore leggibilità
#Ordino il DataFrame in base alle colonne 'priorità', 'unit_id', e 'document_index'
confidenza_media = confidenza_media.sort_values(by=['unit_id', 'document_index']).reset_index(drop=True)

display(confidenza_media)

In [ ]:
#Salvo il tutto in una cartella che andiamo a creare
cartella_output = r'C:\Users\Gatti\Desktop\ProgettoSocial\Risultato_Analisi\media'

#Creo la cartella se non esiste già
if not os.path.exists(cartella_output):
    os.makedirs(cartella_output)

#Salvo il DataFrame in formato CSV nella cartella di output
confidenza_media.to_csv(os.path.join(cartella_output, 'confidenza_media.csv'))
tempo_medio.to_csv(os.path.join(cartella_output, 'tempo_medio.csv'))

4.4.f

1.Quali sono le caratteristiche demografiche e culturali dei lavoratori che potete
dedurre dal questionario?

In [ ]:
display(workers_questionnaire)

In [ ]:
'''#Creazio il grafico a torta per la colonna 'age'
plt.figure(figsize=(8, 8))
workers_questionnaire['age'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('Distribuzione per età')
plt.show()

#Creazio il grafico a torta per la colonna 'gender'
plt.figure(figsize=(8, 8))
workers_questionnaire['gender'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('Distribuzione per genere')
plt.show()

#Creazio il grafico a torta per la colonna 'education',
plt.figure(figsize=(8, 8))
workers_questionnaire['age'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('Distribuzione per età')
plt.show()

#Creazio il grafico a torta per la colonna 'knowledge'
plt.figure(figsize=(8, 8))
workers_questionnaire['gender'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('Distribuzione per genere')
plt.show()

#Creazio il grafico a torta per la colonna 'frequency'
plt.figure(figsize=(8, 8))
workers_questionnaire['gender'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('Distribuzione per genere')
plt.show()'''

4.4.f

2.Vi è accordo tra ciascun lavoratore? Per quali dimensioni vi è più accordo? Per quali meno?

In [ ]:
print(f'Codice eseguito correttamente')